In [22]:
import os
import boto3
import sys

# Set AWS region
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

# # Import utils (adjust path as needed)
# Add current directory to Python path explicitly
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"Current directory: {current_dir}")
print(f"Python path includes current dir: {current_dir in sys.path}")

# Now try importing
import techresidential_utils as utils
print("✅ utils imported successfully")

Current directory: /Users/hsin-weilin/Desktop/projects/AWS_AgentCore_TechResidential/agents
Python path includes current dir: True
✅ utils imported successfully


In [25]:
# Cognito configuration
USER_POOL_NAME = "techresidential-agentcore--user-pool"
RESOURCE_SERVER_ID = "techresidential-gateway-id"  
RESOURCE_SERVER_NAME = "techresidential-gateway-name"
CLIENT_NAME = "techresidential-agentcore-gateway-client"

SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]

scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

# Initialize Cognito client
cognito = boto3.client("cognito-idp", region_name=REGION)

# Create Cognito resources
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
client_id, client_secret = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)

cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'

print(f"User Pool ID: {user_pool_id}")
print(f"Client ID: {client_id}")
print(f"Discovery URL: {cognito_discovery_url}")

Creating new user pool
Domain created as well
creating new resource server
creating new m2m client
User Pool ID: us-east-1_VKErYTcMn
Client ID: 3cr4rm7rjj8hg5sumhqkhm2g1p
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_VKErYTcMn/.well-known/openid-configuration


In [24]:
# Create IAM role that allows Gateway to invoke your Lambda functions
gateway_iam_role = utils.create_agentcore_gateway_role("techresidential-gateway-role")
gateway_role_arn = gateway_iam_role['Role']['Arn']
print(f"Gateway IAM Role ARN: {gateway_role_arn}")

attaching role policy agentcore-techresidential-gateway-role-role
Gateway IAM Role ARN: arn:aws:iam::254599367545:role/agentcore-techresidential-gateway-role-role


In [26]:
# Initialize Gateway client
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

# Configure Cognito authorizer
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],
        "discoveryUrl": cognito_discovery_url
    }
}

# Create the Gateway
create_response = gateway_client.create_gateway(
    name='TechResidentialAgentcoreGateway',
    roleArn=gateway_role_arn,
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway for TechResidential Lambda functions'
)

gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

print(f"Gateway ID: {gatewayID}")
print(f"Gateway URL: {gatewayURL}")

Gateway ID: techresidentialagentcoregateway-umoj4ednav
Gateway URL: https://techresidentialagentcoregateway-umoj4ednav.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp


In [27]:
# Replace with your actual Lambda ARN
GET_CREDENTIALS_LAMBDA_ARN = "arn:aws:lambda:us-east-1:254599367545:function:agentcore-credential-retrieval"

# lambda target configuration
get_credentials_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": GET_CREDENTIALS_LAMBDA_ARN,
            "toolSchema": {
                "inlinePayload": [
                    {
                        # specify a tool name for the agent to use, a target can have many tools
                        "name": "get_credentials",
                        "description": "Retrieve login credentials and website information for a client",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "client_name": {
                                    "type": "string",
                                    "description": "Name of the client (e.g., TechResidential)"
                                }
                            },
                            "required": ["client_name"]
                            # require client_name in the event to invoke
                        }
                    }
                ]
            }
        }
    }
}

# gateway IAM role will be passed on to this target
get_credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

# create a gateway lambda target
credentials_target = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name='CredentialsRetriever',
    description='Lambda target for retrieving client credentials',
    targetConfiguration=get_credentials_target_config,
    credentialProviderConfigurations=get_credential_config
)

print("Get_Credentials target created successfully")

Get_Credentials target created successfully


In [28]:
# Replace with your actual Lambda ARN
GET_S3_PROMPTFILE_LAMBDA_ARN = "arn:aws:lambda:us-east-1:254599367545:function:agentcore-prompt-file-retrieval"

get_prompt_file_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": GET_S3_PROMPTFILE_LAMBDA_ARN,
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_prompt_file",
                        "description": "Retrieve prompt file from S3 for a client",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "client_name": {
                                    "type": "string",
                                    "description": "Name of the client (e.g., TechResidential)"
                                }
                            },
                            "required": ["client_name"]
                        }
                    }
                ]
            }
        }
    }
}

s3_target = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name='PromptFileRetriever',
    description='Lambda target for retrieving client prompt files from S3',
    targetConfiguration=get_prompt_file_target_config,
    credentialProviderConfigurations=get_credential_config
)

print("S3 file retriever target created successfully")

S3 file retriever target created successfully


In [29]:
print(client_secret)

105ld6eu7n12l6hmbu3dtmin3ea0ehse7hmgo2v3b1pusqmgkajr


In [30]:
import time
time.sleep(10)  # Wait for domain propagation. This is necessary because Cognito is a global service and clients (agents, browsers, SDKs) from any location need to be able to reach the domain for authentication and token requests.

print("Requesting access token from Cognito...")
token_response = utils.get_token(user_pool_id, client_id, client_secret, scopeString, REGION)
token = token_response["access_token"]
print("Access token obtained successfully")

Requesting access token from Cognito...
3cr4rm7rjj8hg5sumhqkhm2g1p
Access token obtained successfully


In [31]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)
logging.basicConfig(format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()])

# Create MCP client
def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL, headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

# Initialize Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
)

# Test the integration
with client:
    # List available tools
    tools = client.list_tools_sync()
    print(f"Available tools: {[tool.name for tool in tools]}")
    
    # Create agent with tools
    agent = Agent(model=model, tools=tools)
    
    # Test credentials retrieval
    print("\n=== Testing Credentials Retrieval ===")
    response = agent("Get the credentials for client TechResidential")
    print(response)
    
    # Test prompt file retrieval
    print("\n=== Testing Prompt File Retrieval ===")
    response = agent("Get the prompt file for client TechResidential")
    print(response)
    
    # Test direct tool calls
    print("\n=== Direct Tool Call - Credentials ===")
    result = client.call_tool_sync(
        tool_use_id="creds-call-1",
        name="CredentialsRetriever___get_credentials",
        arguments={"client_name": "TechResidential"}
    )
    print(f"Credentials result: {result['content'][0]['text']}")
    
    print("\n=== Direct Tool Call - Prompt File ===")
    result = client.call_tool_sync(
        tool_use_id="prompt-call-1", 
        name="PromptFileRetriever___get_prompt_file",
        arguments={"client_name": "TechResidential"}
    )
    print(f"Prompt file result: {result['content'][0]['text']}")

AttributeError: 'MCPAgentTool' object has no attribute 'name'